In [1]:
import pandas as pd
import numpy as np
import sys, os

from matplotlib import pyplot as plt
sys.path.insert(1, os.path.join(sys.path[0], '../../'))  # for importing local packages from src
sys.path.insert(1, os.path.join(sys.path[0], '../'))  # for importing model config

%reload_ext autoreload
%autoreload 2 

## Datasets

In [2]:
from src.model.rc_dataset import ExperimentDataset

DATASET_PATH = '../datasets/hydroshoot_large_trimmed.csv'

dataset = ExperimentDataset(csv_path=DATASET_PATH)

## Model Configuration

In [3]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupKFold
from src.learning.scorers import nmse_scorer
from pipeline_hydroshoot import GroupGenerator, TimeGenerator
from pipeline_base import TrainTestSplitter, DirectTransform, WarmupTransform

# constants
DATASET_ID = 'HydroShoot_large'
STATE_SIZE = 32
N_STATE_SAMPLES = 16
state_ids = None


# Reservoir generation
np.random.seed(42)
generate_state_sample = lambda : np.random.choice(dataset.state_size(), size=STATE_SIZE, replace=False)


# readout model
readout_model = Pipeline([
  ('ridge_regression', Ridge(alpha=1, fit_intercept=True))
])
model_param_grid = [{
  'ridge_regression__alpha': 10 ** np.linspace(np.log10(1e-4), np.log10(1e2), 50)
}]


# Regression task pipeline
datasets = (dataset,)
run_ids = dataset.get_run_ids()
shared_pipeline_params = {
  # Data generation
  'datasets': datasets,
  'groups': GroupGenerator(day_length=24, run_ids=run_ids, days_between_runs=1),
  'time': TimeGenerator(day_length=24, run_ids=run_ids),
  
  # Model training and validation
  'readout_model': readout_model,
  'model_param_grid': model_param_grid,
  'model_scorer': nmse_scorer,
  'folds': GroupKFold(n_splits=5),
  'train_test_split': TrainTestSplitter(block_size=4, test_ratio=0.5),
}

## Regression Pipelines

In [4]:
from pipeline_base import (
  RCPipeline, 
  Rescale, 
  DaylightMask
)

from pipeline_hydroshoot import (
  TargetGenerator, 
  SingleReservoirGenerator,
  MultiReservoirGenerator,
  TargetReservoirGenerator,
  GroupRescale
)


from model_config_hydroshoot import (
  targets, 
  measurable_reservoirs,
  baseline_reservoirs,
  heterogeneous_reservoirs, 
)

### Computational Benchmarks

In [5]:
from pipeline_base import DelayLineTransform, CustomWarmupTransform
from pipeline_base import PolynomialTargetTransform
from pipeline_base import NarmaTargetTransform


def delay_line_benchmark(target_gen, target_id, res_gen, res_id, rescale, *, delay_steps: [int]):
  for d in delay_steps:
    pipeline = RCPipeline(
      metadata={
        'target_id': target_id, 
        'reservoir_id': res_id, 
        'dataset_id': DATASET_ID,
        'benchmark': 'delay_line',
        'delay': d,
      },
      # Data generation
      target=target_gen,
      reservoir=res_gen,
      # Data transformation
      transforms=[
        # NOTE: remove warmup before transform, otherwise the 
        # warmup steps are used as delayed prediction targets.
        WarmupTransform(warmup_days=4, day_length=24),
        DelayLineTransform(delay_steps=d),
      ],
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        rescale
      ],
      **shared_pipeline_params,
    )
    yield pipeline


def polynomial_benchmark(target_gen, target_id, res_gen, res_id, rescale, *, exponents: [int]):
  for e in exponents:
    pipeline = RCPipeline(
      metadata={
        'target_id': target_id, 
        'reservoir_id': res_id, 
        'dataset_id': DATASET_ID,
        'benchmark': 'polynomial',
        'exponent': e,
      },
      # Data generation
      target=target_gen,
      reservoir=res_gen,
      # Data transformation
      transforms=[
        WarmupTransform(warmup_days=4, day_length=24),
        PolynomialTargetTransform(poly_coefs=[0] * e + [1]),
      ],
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        rescale
      ],
      **shared_pipeline_params,
    )
    yield pipeline


def narma_benchmark(target_gen, target_id, res_gen, res_id, rescale, *, n_values: [int]):
  for n in n_values:
    pipeline = RCPipeline(
      metadata={
        'target_id': target_id, 
        'reservoir_id': res_id, 
        'dataset_id': DATASET_ID,
        'benchmark': 'NARMA',
        'narma_n': n,
      },
      # Data generation
      target=target_gen,
      reservoir=res_gen,
      # Data transformation
      transforms=[
        # NOTE: remove warmup before transform, otherwise the 
        # warmup steps are used as input to the NARMA system.
        WarmupTransform(warmup_days=4, day_length=24),  
        NarmaTargetTransform(n=n, scale=1),
      ],
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        rescale
      ],
      **shared_pipeline_params,
    )
    yield pipeline

In [12]:
benchmarks = [
  {
    'benchmark': delay_line_benchmark,
    'kwargs': {
      'delay_steps': np.arange(0, 9)  # 0-8 hours
    }
  },
  {
    'benchmark': polynomial_benchmark,
    'kwargs': {
      'exponents': np.arange(1, 10)  # exponents 1-9
    }
  },
  {
    'benchmark': narma_benchmark,
    'kwargs': {
      'n_values': [2, 4, 8, 12, 18, 24]
      # 'n_values': [2, 4, 6, 8, 10, 12, 18, 20, 24]
    }
  },
]

In [7]:
def generate_benchmark_pipelines(target_gen, target_id, reservoir_gen, reservoir_id, rescale):
  pipeline_generators = []
  for benchmark in benchmarks:
    gen = benchmark['benchmark'](target_gen, target_id, reservoir_gen, reservoir_id, rescale, **benchmark['kwargs'])
    pipeline_generators.append(gen)
  return chain(*pipeline_generators)

In [8]:
target_state_pairs = [(target, state_var) for target in targets for state_var in measurable_reservoirs]
target_env_pairs = [(name, target, env_targets) for target in targets for (name, env_targets) in baseline_reservoirs]
target_het_pairs = [(name, target, state_vars) for target in targets for (name, state_vars) in heterogeneous_reservoirs]

print(targets)
print(measurable_reservoirs)
print(baseline_reservoirs)
print(heterogeneous_reservoirs)

['input_Tac', 'input_u', 'input_hs', 'input_Rg', 'output_Rg', 'output_An', 'output_E', 'output_Tleaf']
['state_An', 'state_Tlc', 'state_gs', 'state_E', 'state_Flux', 'state_psi_head']
[('env_all', ['input_Tac', 'input_u', 'input_hs', 'input_Rg']), ('env_temp', ['input_Tac']), ('env_humidity', ['input_hs']), ('env_PAR', ['input_Rg']), ('env_wind', ['input_u'])]
[('state_all', ('state_An', 'state_Tlc', 'state_gs', 'state_E', 'state_Flux', 'state_psi_head'))]


In [13]:
from itertools import chain


def generate_pipelines(state_ids):
  all_pipelines = []

  for target, state_var in target_state_pairs:
    target_gen = TargetGenerator(target=target, run_ids=run_ids)
    res_gen = SingleReservoirGenerator(state_var=state_var, run_ids=run_ids, state_ids=state_ids)
    pipelines = generate_benchmark_pipelines(
      target_gen, target, res_gen, state_var, 
      Rescale(per_feature=False)
    )
    all_pipelines += pipelines

  for name, target, state_vars in target_het_pairs:
    target_gen = TargetGenerator(target=target, run_ids=run_ids)
    res_gen = MultiReservoirGenerator(state_vars=state_vars, run_ids=run_ids, state_ids=state_ids)
    pipelines = generate_benchmark_pipelines(
      target_gen, target, res_gen, name, 
      GroupRescale(datasets=datasets, state_vars=state_vars)
    )
    all_pipelines += pipelines

  for name, target, env_targets in target_env_pairs:
    target_gen = TargetGenerator(target=target, run_ids=run_ids)
    res_gen = TargetReservoirGenerator(targets=env_targets, run_ids=run_ids)
    pipelines = generate_benchmark_pipelines(
      target_gen, target, res_gen, name, 
      Rescale(per_feature=True)
    )
    all_pipelines += pipelines

  return all_pipelines


np.random.seed(42)
state_ids = generate_state_sample()
pipelines = generate_pipelines(state_ids)
print(f'Total pipelines: {len(pipelines)}')

Total pipelines: 2304


## Execution

In [15]:
pipes = generate_pipelines(state_ids)
n_pipelines = len(pipes)
print(f'n pipelines: {n_pipelines}')
print(f'n runs: {n_pipelines * N_STATE_SAMPLES}')

n pipelines: 2304
n runs: 36864


In [16]:
from tqdm import tqdm
from itertools import chain
from pipeline_base import execute_pipeline


RESULTS_FILE = 'results_hydroshoot_comp_bench_16sample.csv'

np.random.seed(42)
all_state_samples = [generate_state_sample() for _ in range(N_STATE_SAMPLES)]

total_loops = n_pipelines * N_STATE_SAMPLES
results = []


with tqdm(total=total_loops) as pbar:

  for i_state_sample, state_ids in enumerate(all_state_samples):
    pipelines = generate_pipelines(state_ids)
    for pipeline in pipelines:
      result = execute_pipeline(pipeline)
      result['state_sample'] = i_state_sample
      results.append(result)
      pbar.update(1)


results_df = pd.DataFrame.from_dict(results)
results_df.to_csv(RESULTS_FILE)
print(f'Saved scores to {RESULTS_FILE}')

  2%|▏         | 656/36864 [03:14<2:59:05,  3.37it/s]


KeyboardInterrupt: 